# $\chi^2$ -  Hyphotesis Testing

## Introduction and Formulation

The $\chi^2$ distribution is the result of adding independent standard normal random variables. Let $Z_i$ from $i = 1 \dots n$ normal independent random variables. Define
$$ Q = \sum_{i=1}^{n} Z_i^2 $$
Then we say that $Q \sim \chi_n^2$ whit $n$ degrees of freedom. 

## Hyphotesis Testing 

This distribution is widely used for Hyphotesis Testing. Suppose that I have a survey with $X = (X_1 , \dots, X_n)$ observations of $X \sim Bernoulli(p)$ (Null Hyphotesis). 

 - The real number of observations of class 1 is $\sum_{i=1}^n X_i = o_1$ and $o_0 = n - o_1$.
 - The expected number of observations is $m_1 = np$ and $m_0 = n(1-p)$ (under the null hyphotesis)

If $n$ is a big number, then by TCL $O_1 = \sum_{i=1}^n X_i \sim \mathcal{N}(np , np(1-p))$, in other words,  
$$ \frac{\sum_{i=1}^{n}X_i - np}{\sqrt{np(1-p)}} \sim \mathcal{N}(0, 1)$$

Raised by 2, we get

$$ \frac{(\sum_{i=1}^{n}X_i - np)^2}{np(1-p)} \sim \chi^2_{1}$$

Rewriting this expression for the Bernoulli case we get 

$$ \frac{((n-\sum_{i=1}^n X_i) - n(1-p))^2}{n(1-p)} + \frac{(\sum_{i=1}^n X_i - np)^2}{np} \sim \chi^2_{1}$$

Replacing with the variable definitions above we get 

$$ \frac{(o_0 - m_0)^2}{m_0} + \frac{(o_1 - m_1)^2}{m_1} \sim \chi^2_{1} $$ 

And for the general case with $k$ classes the result stands for the next formula 

$$ \sum_{j=1}^{k}\frac{(o_j - m_j)^2}{m_j} \sim \chi^2_{k-1}$$ 



## Implementation



In [23]:
import numpy as np
import pandas as pd 

# library for chi2 testing 
from scipy.stats import chi2 
from scipy.stats import chi2_contingency

Suppose that I have an AB Testing with the next results: 

In [35]:
index = ["clicked", "not_clicked"]
a_variant_obs = [70, 140]
b_variant_obs = [20, 80]

print("Observed Results")
results_obs = pd.DataFrame()
results_obs["action"] = index
results_obs.set_index("action", inplace = True) 
results_obs["variant A"] = a_variant_obs
results_obs["variant B"] = b_variant_obs
results_obs

Observed Results


,variant A,variant B
action,,
clicked,70,20
not_clicked,140,80


### Manual Calculation

In [36]:
variant_total = results_obs.sum(axis = 0)
variant_total

variant A    210
variant B    100
dtype: int64

In [37]:
click_total = results_obs.sum(axis = 1)
click_total

action
clicked         90
not_clicked    220
dtype: int64

In [38]:
# Calculate the expected number of clicks 
print("Expected Results")
results_exp = pd.DataFrame()
results_exp["action"] = index 
results_exp.set_index("action", inplace = True)
results_exp["variant A"] = click_total / click_total.sum() * variant_total["variant A"]
results_exp["variant B"] = click_total / click_total.sum() * variant_total["variant B"]
results_exp

Expected Results


,variant A,variant B
action,,
clicked,60.967742,29.032258
not_clicked,149.032258,70.967742


In [39]:
# Calculate the square difference
print("Square Difference")
squared_difference = (results_obs - results_exp)**2 / results_exp
squared_difference

Square Difference


,variant A,variant B
action,,
clicked,1.338112,2.810036
not_clicked,0.547410,1.149560


In [40]:
chi_2_stat = squared_difference.sum().sum()
print(f"Chi2 Statistic Value: {chi_2_stat}")

Chi2 Statistic Value: 5.845117845117846


Now, we calculate the p value using this Chi2 statistic

In [41]:
# Degrees of Freedom 
dof = (results_obs.shape[0] - 1) * (results_obs.shape[1] - 1)
print(f"Degrees of Freedom: {dof}")

Degrees of Freedom: 1


In [42]:
# Use the Survival function of (1-cdf) to get the p-value
p_value = chi2.sf(chi_2_stat, dof)
print(f"The p value is: {p_value}")

The p value is: 0.015620327818585281


In [43]:
p_value_threshold = 0.05
if p_value < p_value_threshold: 
    print("The null hyphotesis is rejected")
else: 
    print("Not enough evidence to reject the null hyphotesis")

The null hyphotesis is rejected


### Using Scipy

In [44]:
test_results = chi2_contingency(results_obs, correction=False)
test_results

Chi2ContingencyResult(statistic=5.845117845117845, pvalue=0.015620327818585326, dof=1, expected_freq=array([[ 60.96774194,  29.03225806],
       [149.03225806,  70.96774194]]))

In [45]:
p_value_threshold = 0.05
if test_results.pvalue < p_value_threshold: 
    print("The null hyphotesis is rejected")
else: 
    print("Not enough evidence to reject the null hyphotesis")


The null hyphotesis is rejected
